In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pickle
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
import glob
import csv


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from matplotlib import pylab as plt
%matplotlib inline

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_colwidth", 5000)
pd.set_option("display.max_columns", 2000)


DATA_DIR = '../data/'

/var/folders/zh/c52bnb1524xfzk_885ccgt4w0000gq/T/ipykernel_9629/3228527578.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
glob.glob(f"{DATA_DIR}*")

['../data/NYT_Russia_Ukraine.csv',
 '../data/Ukraine_troops.csv',
 '../data/Guardians_Russia_Ukraine.csv',
 '../data/Ukraine_war.csv',
 '../data/Russia_invade.csv',
 '../data/russian_invasion_of_ukraine.csv',
 '../data/Ukraine_nato.csv',
 '../data/Russian_troops.csv',
 '../data/StandWithUkraine.csv',
 '../data/Ukraine_border.csv',
 '../data/Russian_border_Ukraine.csv']

# Load data

In [4]:
import pickle

data_tweet_clean = pickle.load(open('data_filter.pkl', 'rb'))
data_tweet_clean.shape

(117110, 31)

In [35]:
data_tweet_clean.id.to_csv('filter_id.csv', index=False)

# Text preprocessing

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Convert to list
data = data_tweet_clean.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Its all on Putin! #WarCriminalPutin #PutinWarCrimes #PutinIsaWarCriminal '
 'Putin invaded Ukraine uninvited and not desired. No one wants a traitor that '
 'arrested children over flowers! Or beats the Russian people for dissenting '
 'him! Take back your troops from Ukraine!']


In [15]:
import string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


#remove hashtags
data = [strip_all_entities(strip_links(sent)) for sent in data]

pprint(data[:1])

['Its all on Putin Putin invaded Ukraine uninvited and not desired No one '
 'wants a traitor that arrested children over flowers Or beats the Russian '
 'people for dissenting him Take back your troops from Ukraine']


In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['its', 'all', 'on', 'putin', 'putin', 'invaded', 'ukraine', 'uninvited', 'and', 'not', 'desired', 'no', 'one', 'wants', 'traitor', 'that', 'arrested', 'children', 'over', 'flowers', 'or', 'beats', 'the', 'russian', 'people', 'for', 'dissenting', 'him', 'take', 'back', 'your', 'troops', 'from', 'ukraine']]


In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['its', 'all', 'on', 'putin', 'putin', 'invaded', 'ukraine', 'uninvited', 'and', 'not', 'desired', 'no', 'one', 'wants', 'traitor', 'that', 'arrested', 'children', 'over', 'flowers', 'or', 'beats', 'the', 'russian', 'people', 'for', 'dissenting', 'him', 'take', 'back', 'your', 'troops', 'from', 'ukraine']


In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['invade', 'desire', 'want', 'traitor', 'arrest', 'child', 'flower', 'beat', 'russian', 'people', 'dissent', 'take', 'back', 'troop', 'ukraine']]


In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


# Build 20 topics extraction model

In [30]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.433*"border" + 0.032*"tension" + 0.024*"cross" + 0.022*"war" + '
  '0.018*"video" + 0.014*"people" + 0.014*"buildup" + 0.013*"make" + '
  '0.011*"urge" + 0.009*"weapon"'),
 (1,
  '0.108*"show" + 0.103*"crisis" + 0.083*"unit" + 0.061*"guard" + '
  '0.045*"equipment" + 0.028*"large" + 0.028*"watch" + 0.027*"look" + '
  '0.026*"missiles_advance" + 0.026*"happen"'),
 (2,
  '0.094*"build" + 0.070*"state" + 0.058*"come" + 0.053*"belarusian" + '
  '0.046*"also" + 0.046*"time" + 0.034*"defend" + 0.030*"way" + 0.029*"big" + '
  '0.028*"aggression"'),
 (3,
  '0.184*"russian" + 0.066*"ukraine" + 0.051*"military" + 0.044*"force" + '
  '0.041*"say" + 0.033*"ukrainian" + 0.029*"troop" + 0.026*"invasion" + '
  '0.019*"move" + 0.017*"attack"'),
 (4,
  '0.139*"belarus" + 0.135*"take" + 0.042*"include" + 0.037*"president" + '
  '0.037*"accord" + 0.033*"helicopter" + 0.033*"flee" + 0.025*"source" + '
  '0.024*"remove" + 0.022*"read"'),
 (5,
  '0.051*"country" + 0.041*"invade" + 0.034*"want" + 0

In [31]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Perplexity:  -18.273007475627555

# Coherence Score:  0.24473090500373823


Perplexity:  -8.052834510097568

Coherence Score:  0.27603964573674084


In [32]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/dzikr/mambaforge/envs/research/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.299731 -0.324932       1        1  32.560935
0      0.309448  0.340314       2        1  18.180278
5     -0.017832 -0.085587       3        1  11.459441
7     -0.026204 -0.039746       4        1   9.477455
8     -0.090265 -0.023172       5        1   5.971857
6     -0.076538  0.023374       6        1   5.127982
9     -0.102505  0.022940       7        1   4.890384
1     -0.085944  0.042314       8        1   4.755837
2     -0.097427  0.013596       9        1   4.521813
4     -0.112464  0.030898      10        1   3.054018, topic_info=                   Term          Freq         Total Category  logprob  loglift
66               border  96018.000000  96018.000000  Default  30.0000  30.0000
9               russian  73265.000000  73265.000000  Default  29.0000  29.0000
13              ukraine  26198.000000  26198.000000  Default  28.0000  28.0000
305                talk   8151.000000   8151.000000  Default  27.0000  27.0000
70             military  20330.000000  20330.000000  Default  26.0000  26.0000
246               force  17638.000000  17638.000000  Default  25.0000  25.0000
228                show   6244.000000   6244.000000  Default  24.0000  24.0000
2321             crisis   5970.000000   5970.000000  Default  23.0000  23.0000
581             belarus   5186.000000   5186.000000  Default  22.0000  22.0000
53                  say  16142.000000  16142.000000  Default  21.0000  21.0000
10                 take   5018.000000   5018.000000  Default  20.0000  20.0000
223                  go   5793.000000   5793.000000  Default  19.0000  19.0000
685               build   5213.000000   5213.000000  Default  18.0000  18.0000
79              country   7160.000000   7160.000000  Default  17.0000  17.0000
133           ukrainian  13310.000000  13310.000000  Default  16.0000  16.0000
469                unit   4789.000000   4789.000000  Default  15.0000  15.0000
607                warn   4537.000000   4537.000000  Default  14.0000  14.0000
121               close   5619.000000   5619.000000  Default  13.0000  13.0000
12                troop  11616.000000  11616.000000  Default  12.0000  12.0000
7                invade   5716.000000   5716.000000  Default  11.0000  11.0000
259               state   3879.000000   3879.000000  Default  10.0000  10.0000
2233            tension   7026.000000   7026.000000  Default   9.0000   9.0000
42                think   4104.000000   4104.000000  Default   8.0000   8.0000
33             invasion  10229.000000  10229.000000  Default   7.0000   7.0000
833               guard   3567.000000   3567.000000  Default   6.0000   6.0000
14                 want   4751.000000   4751.000000  Default   5.0000   5.0000
253                come   3183.000000   3183.000000  Default   4.0000   4.0000
47                  get   3309.000000   3309.000000  Default   3.0000   3.0000
67                cross   5339.000000   5339.000000  Default   2.0000   2.0000
567          belarusian   2906.000000   2906.000000  Default   1.0000   1.0000
9               russian  73264.960105  73265.859842   Topic1  -1.6907   1.1220
13              ukraine  26197.338022  26198.237631   Topic1  -2.7192   1.1220
70             military  20329.474244  20330.373922   Topic1  -2.9728   1.1220
246               force  17637.729078  17638.628708   Topic1  -3.1148   1.1220
53                  say  16141.238619  16142.138215   Topic1  -3.2035   1.1220
133           ukrainian  13309.321768  13310.221405   Topic1  -3.3964   1.1220
12                troop  11615.174490  11616.074176   Topic1  -3.5325   1.1220
33             invasion  10228.654723  10229.554400   Topic1  -3.6596   1.1220
414                move   7386.921532   7387.821152   Topic1  -3.9851   1.1219
65               attack   6797.548105   6798.447751   Topic1  -4.0683   1.1219
86                 tank   5058.744731   5059.644352   Topic1  -4.3637 